In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()
import kglab
import pandas as pd

/afs/crc.nd.edu/group/TAI/Users/painswor/sbom-analysis/.venv/lib/python3.9/site-packages/nbdev/export.py:54: UserWarning: Notebook '/afs/crc.nd.edu/group/TAI/Users/painswor/sbom-analysis/nbs/00_core.ipynb' uses `#|export` without `#|default_exp` cell.
Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.
See https://nbdev.fast.ai/getting_started.html for more information.
  warn(f"Notebook '{nbname}' uses `#|export` without `#|default_exp` cell.\n"


# SBOM Analysis Core Functions

This page defines all boilerplate functions to be used when analyzing sboms.  To see these functions in action please view the following pages.

#| hide
### Load KG for testing

In [ ]:
import kglab
kg = kglab.KnowledgeGraph()
kg.load_rdf("sboms/rdf/model.rdf.xml", format="xml")

<kglab.kglab.KnowledgeGraph>

## Querying

In [ ]:
#| export
def show_metadata(kg, dataframe=True):
    """
    Return total number of triples, distict entities, and properties to a pandas dataframe.
    Print if df=false
    """
    
    query = """
    SELECT 
        (COUNT(*) AS ?triples)
        (COUNT(DISTINCT ?entity) AS ?entities)
        (COUNT(DISTINCT ?property) AS ?properties)
    WHERE {
        ?subject ?property ?object .
        BIND(?subject AS ?entity) .
    }
    """

    if dataframe:
        return kg.query_as_df(query)

    for row in kg.query(query):
        print("Total Triples:", row['triples'])
        print("Distinct Entities:", row['entities'])
        print("Distinct Properties:", row['properties'])

In [ ]:
#| hide
show_metadata(kg, dataframe=False)

Total Triples: 306
Distinct Entities: 56
Distinct Properties: 62


**Number of Distinct Entities**

In [ ]:
#| echo: false
query = """
SELECT (COUNT(DISTINCT ?entity) as ?count)
WHERE {
  ?entity a ?type .
}
"""

result = kg.query(query)
for row in result:
    print("Distinct Entities:", row["count"])


Distinct Entities: 56


**Number of Distinct Properties**

In [ ]:
#| echo: false
query = """
SELECT (COUNT(DISTINCT ?property) as ?count)
WHERE {
  ?s ?property ?o .
}
"""

result = kg.query(query)
for row in result:
    print("Distinct Properties:", row["count"])

Distinct Properties: 62


**Number of Entities per Type**

In [ ]:
#| echo: false
query = """
SELECT ?type (COUNT(?entity) as ?count)
WHERE {
  ?entity a ?type .
}
GROUP BY ?type
ORDER BY DESC(?count)
"""

result = kg.query(query)
for row in result:
    print(row["type"], ":", row["count"])


http://spdx.org/rdf/terms#Relationship : 11
http://spdx.org/rdf/terms#Checksum : 10
http://spdx.org/rdf/terms#ExtractedLicensingInfo : 5
http://spdx.org/rdf/terms#Annotation : 5
http://spdx.org/rdf/terms#Package : 4
http://spdx.org/rdf/terms#File : 4
http://spdx.org/rdf/terms#ExternalRef : 3
http://spdx.org/rdf/terms#DisjunctiveLicenseSet : 2
http://www.w3.org/2009/pointers#StartEndPointer : 2
http://www.w3.org/2009/pointers#ByteOffsetPointer : 2
http://www.w3.org/2009/pointers#LineCharPointer : 2
http://spdx.org/rdf/terms#SpdxDocument : 1
http://spdx.org/rdf/terms#PackageVerificationCode : 1
http://spdx.org/rdf/terms#ConjunctiveLicenseSet : 1
http://spdx.org/rdf/terms#CreationInfo : 1
http://spdx.org/rdf/terms#ExternalDocumentRef : 1
http://spdx.org/rdf/terms#Snippet : 1


**Top N Properties by Popularity**

In [ ]:
#| echo: false
N = 10

query = """
SELECT ?property (COUNT(?property) as ?count)
WHERE {
  ?s ?property ?o .
}
GROUP BY ?property
ORDER BY DESC(?count)
LIMIT %d
""" % N

result = kg.query(query)
for row in result:
    print(row["property"], ":", row["count"])


http://www.w3.org/1999/02/22-rdf-syntax-ns#type : 56
http://www.w3.org/2000/01/rdf-schema#comment : 14
http://spdx.org/rdf/terms#relatedSpdxElement : 11
http://spdx.org/rdf/terms#relationship : 11
http://spdx.org/rdf/terms#relationshipType : 11
http://spdx.org/rdf/terms#fileContributor : 11
http://spdx.org/rdf/terms#checksumValue : 10
http://spdx.org/rdf/terms#checksum : 10
http://spdx.org/rdf/terms#algorithm : 10
http://spdx.org/rdf/terms#licenseConcluded : 9


In [ ]:
#| echo: false
measure = kglab.Measure()
measure.measure_graph(kg)

print("edges", measure.get_edge_count())
print("nodes", measure.get_node_count())

edges 306
nodes 99


## Files

Contained file schema

In [ ]:
#| echo: false
query = """
PREFIX spdx:<http://spdx.org/rdf/terms#>
SELECT DISTINCT ?property
WHERE {
  ?file rdf:type spdx:File .
  ?file ?property ?value .
}
"""


df = kg.query_as_df(query)
df

,property
0,rdf:type
1,spdx:licenseInfoInFile
2,rdfs:comment
3,spdx:licenseConcluded
4,spdx:checksum
5,spdx:fileContributor
6,spdx:fileType
7,spdx:relationship
8,spdx:licenseComments
9,spdx:fileName


In [ ]:
namespaces = {
    "spdx": "http://spdx.org/rdf/terms#"
}
subgraph = kglab.KnowledgeGraph(
    namespaces = namespaces,
)
query = """
PREFIX spdx:<http://spdx.org/rdf/terms#>
CONSTRUCT {
    ?file rdf:type spdx:File .
    ?file spdx:fileName ?fileName .
    ?file spdx:fileContributor ?contributor .
    ?file spdx:licenseInfoInFile ?licenseInFile .
    ?file spdx:checksum ?checksum .
    ?file spdx:relationship ?relationship .
}
WHERE {
    ?file rdf:type spdx:File .
    ?file spdx:fileName ?fileName .
    ?file spdx:fileContributor ?contributor .
    ?file spdx:licenseInfoInFile ?licenseInFile .
    OPTIONAL {?file spdx:relationship ?relationship .}
}
"""

for row in kg.query(query):
    s, p, o = row
    subgraph.add(s, p, o)
    
VIS_STYLE = {
    "spdx": {
        "color": "orange",
        "size": 40,
    },
    "rdf":{
        "color": "red",
        "size": 30,
    },
}
g = kglab.SubgraphTensor(subgraph)
pyvis_graph = g.build_pyvis_graph(notebook=True, style=VIS_STYLE)
pyvis_graph.force_atlas_2based()
pyvis_graph.show("tmp.fig.html")

tmp.fig.html


In [ ]:
query = """
PREFIX spdx:<http://spdx.org/rdf/terms#>
SELECT 
(?file AS ?fileID)
?fileName
?fileType
?licenseInFile
(GROUP_CONCAT(?contributor; SEPARATOR=", ") AS ?contributors)
(GROUP_CONCAT(?licenseConcluded; SEPARATOR=", ") AS ?licenseConcluded)
?checksum
?relationship
?annotation
?comment
?licenseComments
?noticeText
WHERE {
  ?file rdf:type spdx:File .
  ?file spdx:fileName ?fileName .
  ?file spdx:fileContributor ?contributor .
  ?file spdx:licenseInfoInFile ?licenseInFile .
  ?file spdx:licenseConcluded ?licenseConcluded .
  ?file spdx:checksum ?checksum .
  ?file spdx:copyrightText ?copyrightText .
  ?file spdx:fileType ?fileType .
  OPTIONAL {?file spdx:relationship ?relationship .}
  OPTIONAL {?file spdx:annotation ?annotation . }
  OPTIONAL {?file rdfs:comment ?comment . }
  OPTIONAL {?file spdx:licenseComments ?licenseComments . }
  OPTIONAL {?file spdx:noticeText ?noticeText . }
}
GROUP BY ?file
"""

kg.query_as_df(query)

,fileID,fileName,fileType,licenseInFile,contributors,licenseConcluded,checksum,relationship,comment,licenseComments,noticeText,annotation
0,<http://spdx.org/spdxdocs/spdx-example-444504E...,./lib-source/jena-2.6.3-sources.jar,spdx:fileType_archive,<http://spdx.org/spdxdocs/spdx-example-444504E...,"Apache Software Foundation, Hewlett Packard Inc.",http://spdx.org/spdxdocs/spdx-example-444504E0...,_:N71af9f1e08c749c3aba7184c9952d6d0,_:N6216ca7b5f8449ada139bf524e1e9b95,This file belongs to Jena,This license is used by Jena,NaN,NaN
1,<http://spdx.org/spdxdocs/spdx-example-444504E...,./lib-source/commons-lang3-3.1-sources.jar,spdx:fileType_archive,<http://spdx.org/licenses/Apache-2.0>,Apache Software Foundation,http://spdx.org/licenses/Apache-2.0,_:N097c8b1780fc498fb9b11b97c9f0f21a,_:Ne313a27bca7c4cdfa4f8f250c7e8cb9c,This file is used by Jena,NaN,Apache Commons Lang\nCopyright 2001-2011 The A...,NaN
2,<http://spdx.org/spdxdocs/spdx-example-444504E...,./src/org/spdx/parser/DOAPProject.java,spdx:fileType_source,<http://spdx.org/licenses/Apache-2.0>,"Source Auditor Inc., Black Duck Software In.c,...","http://spdx.org/licenses/Apache-2.0, http://sp...",_:N0c365dd0ba70437298fd6ed1f93ae193,NaN,NaN,NaN,NaN,NaN
3,<http://spdx.org/spdxdocs/spdx-example-444504E...,./package/foo.c,spdx:fileType_source,<http://spdx.org/spdxdocs/spdx-example-444504E...,"IBM Corporation, IBM Corporation, Modified by ...","N32c9a8c7800442cda737a5f2d9bd4b91, N32c9a8c780...",_:N4578f96238794a8f8047c2b9fd49384f,_:N41b7fa671c5a486fa103189ccc6cfaf3,The concluded license was taken from the packa...,The concluded license was taken from the packa...,Copyright (c) 2001 Aaron Lehmann aaroni@vitelu...,_:N84d0424f37054f078883ca28bfd9fd5e


In [ ]:
pyvis_graph = kg.visualize_query(query, notebook=True)

pyvis_graph.force_atlas_2based()
pyvis_graph.show("tmp.fig06.html")

tmp.fig06.html


In [ ]:
query = """
SELECT ?subject ?object
WHERE {
    ?subject rdf:type spdx:File .
    ?subject spdx:relationship ?object .
}
"""

files = kg.query_as_df(query)
files.head(5)

,subject,object
0,<http://spdx.org/spdxdocs/spdx-example-444504E...,_:N2918ecd2880a4c688af16389c67b6952
1,<http://spdx.org/spdxdocs/spdx-example-444504E...,_:N96429b7c2cce4cd1a0c417198c9362de
2,<http://spdx.org/spdxdocs/spdx-example-444504E...,_:N9ecff353b62c42e68b5add0efec8a827


In [ ]:
import networkx as nx

subgraph = kglab.SubgraphMatrix(kg, query)
nx_graph = subgraph.build_nx_graph(nx.DiGraph(), bipartite=True)

In [ ]:
nx.density(nx_graph)

0.1

### Packages

In [ ]:
query = """
PREFIX spdx:<http://spdx.org/rdf/terms#>
SELECT DISTINCT ?property
WHERE {
  ?file rdf:type spdx:Package .
  ?file ?property ?value .
}
order by asc(UCASE(str(?property)))
"""


df = kg.query_as_df(query)
df

,property
0,spdx:annotation
1,spdx:attributionText
2,spdx:checksum
3,spdx:copyrightText
4,spdx:description
5,spdx:downloadLocation
6,spdx:externalRef
7,spdx:filesAnalyzed
8,spdx:licenseComments
9,spdx:licenseConcluded


In [ ]:
query = """
PREFIX spdx:<http://spdx.org/rdf/terms#>
SELECT ?package
       (GROUP_CONCAT(?annotation; SEPARATOR=", ") AS ?annotations)
       (GROUP_CONCAT(?attributionText; SEPARATOR=", ") AS ?attributionTexts)
       ?builtDate
       (GROUP_CONCAT(?checksum; SEPARATOR=", ") AS ?checksums)
       ?comment
       ?copyrightText
       ?description
       ?downloadLocation
       (GROUP_CONCAT(?externalRef; SEPARATOR=", ") AS ?externalRefs)
       ?filesAnalyzed
       (GROUP_CONCAT(?hasFile; SEPARATOR=", ") AS ?hasFiles)
       ?homepage
       ?licenseComments
       ?licenseConcluded
       ?licenseDeclared
       (GROUP_CONCAT(?licenseInfoFromFile; SEPARATOR=", ") AS ?licenseInfoFromFiles)
       ?name
       ?originator
       ?packageFileName
       ?packageVerificationCode
       ?primaryPackagePurpose
       ?releaseDate
       ?sourceInfo
       ?summary
       ?supplier
       ?validUntilDate
       ?versionInfo
       (GROUP_CONCAT(?relationship; SEPARATOR=", ") AS ?relationships)
WHERE {
      ?package rdf:type spdx:Package .
      OPTIONAL { ?package spdx:annotation ?annotation . }
      OPTIONAL { ?package spdx:attributionText ?attributionText . }
      OPTIONAL { ?package spdx:builtDate ?buildDate . }
      OPTIONAL { ?package spdx:checksum ?checksum . }
      OPTIONAL { ?package spdx:comment ?comment . }
      OPTIONAL { ?package spdx:copyrightText ?copyrightText . }
      OPTIONAL { ?package spdx:description ?description . }
      OPTIONAL { ?package spdx:downloadLocation ?downloadLocation . }
      OPTIONAL { ?package spdx:externalRef ?externalRef . }
      OPTIONAL { ?package spdx:filesAnalyed ?filesAnalyzed . }
      OPTIONAL { ?package spdx:hasFile ?hasFile . }
      OPTIONAL { ?package spdx:homepage ?homepage . }
      OPTIONAL { ?package spdx:licenseComments ?licenseComments . }
      OPTIONAL { ?package spdx:licenseConcluded ?licenseConcluded . }
      OPTIONAL { ?package spdx:licenseDeclared ?licenseDeclared . }
      OPTIONAL { ?package spdx:licenseInfoFromFile ?licenseInfoFromFile . }
      OPTIONAL { ?package spdx:name ?name . }
      OPTIONAL { ?package spdx:originator ?originator . }
      OPTIONAL { ?package spdx:packageFileName ?packageFileName . }
      OPTIONAL { ?package spdx:packageVerificationCode ?packageVerificationCode . }
      OPTIONAL { ?package spdx:primaryPackagePurpose ?primaryPackagePurpose . }
      OPTIONAL { ?package spdx:releaseDate ?releaseDate . }
      OPTIONAL { ?package spdx:sourceInfo ?sourceInfo . }
      OPTIONAL { ?package spdx:summary ?summary . }
      OPTIONAL { ?package spdx:supplier ?supplier . }
      OPTIONAL { ?package spdx:validUntilDate ?validUntilDate . }
      OPTIONAL { ?package spdx:versionInfo ?versionInfo . }
      OPTIONAL { ?package spdx:relationship ?relationship . }
}
GROUP BY ?package
"""

pkgs = kg.query_as_df(query)
pkgs.head(5)

,package,annotations,attributionTexts,checksums,copyrightText,downloadLocation,externalRefs,hasFiles,licenseConcluded,licenseDeclared,...,relationships,description,licenseComments,originator,packageFileName,packageVerificationCode,sourceInfo,summary,supplier,versionInfo
0,<http://spdx.org/spdxdocs/spdx-example-444504E...,,,,NOASSERTION,spdx:noassertion,,,spdx:noassertion,spdx:noassertion,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,<http://spdx.org/spdxdocs/spdx-example-444504E...,"N45a4c799f20b4594b0ed2b8b355cad46, N45a4c799f2...",The GNU C Library is free software. See the f...,"N03a2d3d0a5344cc2a1ea34ce3a1296fe, N03a2d3d0a5...",Copyright 2008-2010 John Smith,http://ftp.gnu.org/gnu/glibc/glibc-ports-2.15....,"N1997e51bb6f1435ab4bc1096fddaa9c3, N1997e51bb6...",,_:N87df52a831864bb88e1ea442ae084f0a,_:N7afe9ac63dff4261a951695cbd1f1a6e,...,"N12b259c9947c48d3a30bcd7fca0e8c6d, N552ca3515c...",The GNU C Library defines functions that are s...,The license for this project changed with the ...,Organization: ExampleCodeInspect (contact@exam...,glibc-2.11.1.tar.gz,_:Nba651b8c0b554db8bb1ee1d7ef23024c,uses glibc-2_11-branch from git://sourceware.o...,GNU C library.,Person: Jane Doe (jane.doe@example.com),2.11.1
2,<http://spdx.org/spdxdocs/spdx-example-444504E...,,,Nd133c4b6405c42e7ba92fc6ae3717269,Copyright Saxonica Ltd,https://sourceforge.net/projects/saxon/files/S...,,,<http://spdx.org/licenses/MPL-1.0>,<http://spdx.org/licenses/MPL-1.0>,...,,The Saxon package is a collection of tools for...,Other versions available for a commercial license,NaN,saxonB-8.8.zip,NaN,NaN,NaN,NaN,8.8
3,<http://spdx.org/spdxdocs/spdx-example-444504E...,,,,NOASSERTION,https://search.maven.org/remotecontent?filepat...,N37afccaf22a9473d9eabd2ee323d7d38,,spdx:noassertion,spdx:noassertion,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.12.0


Construct a subgraph from this query

### Relationships

In [ ]:
query = """
PREFIX spdx:<http://spdx.org/rdf/terms#>
SELECT DISTINCT ?property
WHERE {
  ?file rdf:type spdx:Relationship .
  ?file ?property ?value .
}
"""


df = kg.query_as_df(query)
df

,property
0,rdf:type
1,spdx:relationshipType
2,spdx:relatedSpdxElement


In [ ]:
query = """
PREFIX spdx:<http://spdx.org/rdf/terms#>
SELECT ?element ?elementType ?relationshipType ?relatedElement ?relatedElementType
WHERE {
  ?element spdx:relationship ?relationship .
  ?element rdf:type ?elementType .
  ?relationship spdx:relatedSpdxElement ?relatedElement .
  ?relationship spdx:relationshipType ?relationshipType .
  ?relatedElement rdf:type ?relatedElementType .
}
"""

#for row in kg.query(query):
#    print(row.asdict())
#    print()


df = kg.query_as_df(query)
df

,element,elementType,relationshipType,relatedElement,relatedElementType
0,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:File,spdx:relationshipType_contains,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:Package
1,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:Package,spdx:relationshipType_contains,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:File
2,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:Package,spdx:relationshipType_dynamicLink,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:Package
3,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:Package,spdx:relationshipType_contains,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:File
4,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:Package,spdx:relationshipType_contains,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:File
5,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:SpdxDocument,spdx:relationshipType_contains,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:Package
6,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:File,spdx:relationshipType_generatedFrom,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:Package
7,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:SpdxDocument,spdx:relationshipType_describes,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:File
8,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:SpdxDocument,spdx:relationshipType_describes,<http://spdx.org/spdxdocs/spdx-example-444504E...,spdx:Package


In [ ]:
namespaces = {
    "spdx": "http://spdx.org/rdf/terms#",
    "this": "http://spdx.org/spdxdocs/spdx-example-444504E0-4F89-41D3-9A0C-0305E82C3301#"
}
subgraph = kglab.KnowledgeGraph(
    namespaces = namespaces,
)
query = """
PREFIX spdx:<http://spdx.org/rdf/terms#>
CONSTRUCT {
  ?element spdx:relationship ?relationship .
  ?element rdf:type ?elementType .
  ?relationship spdx:relatedSpdxElement ?relatedElement .
  ?relationship spdx:relationshipType ?relationshipType .
  ?relatedElement rdf:type ?relatedElementType .
}
WHERE {
  ?element spdx:relationship ?relationship .
  ?element rdf:type ?elementType .
  ?relationship spdx:relatedSpdxElement ?relatedElement .
  ?relationship spdx:relationshipType ?relationshipType .
  ?relatedElement rdf:type ?relatedElementType .
}
"""

for row in kg.query(query):
    s, p, o = row
    subgraph.add(s, p, o)
    
VIS_STYLE = {
    "spdx": {
        "color": "orange",
        "size": 40,
    },
    "this":{
        "color": "green",
        "size": 30,
    },
}
g = kglab.SubgraphTensor(subgraph)
pyvis_graph = g.build_pyvis_graph(notebook=True, style=VIS_STYLE)
pyvis_graph.force_atlas_2based()
pyvis_graph.show("tmp.fig.html")

tmp.fig.html


## Visualization

In [ ]:
VIS_STYLE = {
    "spdx": {
        "color": "orange",
        "size": 40,
    },
    "rdf":{
        "color": "blue",
        "size": 30,
    },
    "ptr":{
        "color": "red",
        "size": 20,
    },
}

subgraph = kglab.SubgraphTensor(kg)
pyvis_graph = subgraph.build_pyvis_graph(notebook=True, style=VIS_STYLE)

In [ ]:
pyvis_graph.force_atlas_2based()
pyvis_graph.show("tmp.fig03.html")


tmp.fig03.html


In [ ]:
#| default_exp core